# 1. Setting Up Environment




## a. Installing Dependencies


In [1]:
# PyTorch & Torchvision
!pip install --quiet torch torchvision torchaudio

# Roboflow for dataset
!pip install roboflow

# Optuna for Hyperparameter Optimization
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9

## b. Importing Libraries


In [2]:
from roboflow import Roboflow
import os
import torch
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
from torchvision import models
import matplotlib.pyplot as plt
import cv2
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.metrics import (
    confusion_matrix,
    precision_recall_curve,
    classification_report,
    roc_curve,
    auc
)
from sklearn.preprocessing import label_binarize
import optuna
import torch.nn as nn
from torch.utils.data import DataLoader
from time import perf_counter

## c. Download Dataset and Mount Drive

In [3]:
# Download Dataset
rf = Roboflow(api_key="SOWkZCal2FAKPG56WSnb")
project = rf.workspace("work-tqclg").project("tumor-cjxoh")
version = project.version(1)
dataset = version.download("multiclass", location="data")

# Mounting Drive
drive.mount('/content/drive')

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to data in multiclass:: 100%|██████████| 18356/18356 [00:06<00:00, 2963.58it/s]


Mounted at /content/drive


# 2. Model Training

## a. Global Training Configuration

In [16]:
DATA_DIR      = 'data'
TRAIN_IMG_DIR = os.path.join(DATA_DIR, 'train')
VALID_IMG_DIR = os.path.join(DATA_DIR, 'valid')
TEST_IMG_DIR  = os.path.join(DATA_DIR, 'test')
TRAIN_CSV     = os.path.join(TRAIN_IMG_DIR, '_classes.csv')
VALID_CSV     = os.path.join(VALID_IMG_DIR, '_classes.csv')
TEST_CSV      = os.path.join(TEST_IMG_DIR, '_classes.csv')

BATCH_SIZE    = 32
MAX_EPOCHS    = 30
PATIENCE      = 5

LR            = 1e-4
WEIGHT_DECAY  = 1e-3
STEP_SIZE     = 5
GAMMA         = 0.1

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_dir = "/content/drive/MyDrive/Brandon's FYP"
os.makedirs(base_dir, exist_ok=True)
OPTIMIZED_HYPERPARAMETERS = "/content/drive/MyDrive/Brandon's FYP/Hyperparameter Optimization.xlsx"

## b. Model Loader

In [5]:
def get_all_models(num_classes):
    models_list = []

    # ResNet50
    resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
    resnet.fc = torch.nn.Sequential(
        torch.nn.Dropout(0.5),
        torch.nn.Linear(resnet.fc.in_features, num_classes)
    )
    models_list.append(("ResNet50", resnet))

    # EfficientNetB2
    effnet = models.efficientnet_b2(weights=models.EfficientNet_B2_Weights.IMAGENET1K_V1)
    effnet.classifier[1] = torch.nn.Linear(effnet.classifier[1].in_features, num_classes)
    models_list.append(("EfficientNetB2", effnet))

    # MobileNetV3
    mobile = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.IMAGENET1K_V1)
    mobile.classifier[3] = torch.nn.Linear(mobile.classifier[3].in_features, num_classes)
    models_list.append(("MobileNetV3", mobile))

    return models_list


## c. Dataset Class

In [6]:
# ----------------------------
# Custom Dataset Class from CSV
# ----------------------------
class CSVClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.img_dir   = img_dir
        self.transform = transform

        # Load CSV and infer class columns
        df = pd.read_csv(csv_file)
        df.columns = df.columns.str.strip()  # Clean column names
        self.class_names = [c for c in df.columns if c.lower() != 'filename']
        self.num_classes = len(self.class_names)

        self.samples = [
            (row['filename'], int(np.argmax(row[self.class_names].values.astype(int))))
            for _, row in df.iterrows()
        ]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        fname, label = self.samples[idx]
        img_path = os.path.join(self.img_dir, fname)
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.long)

## d. Data Preprocessing (Transformer)

In [7]:
# ----------------------------
# Flexible Preprocessing Function
# mode = 0 → Baseline (resize + normalize only)
# mode = 1 → Enhanced (blur, jitter, flip, rotate, normalize)
# ----------------------------
class GaussianBlur:
    def __init__(self, kernel_size=3):
        self.kernel_size = kernel_size if kernel_size % 2 == 1 else kernel_size + 1

    def __call__(self, img):
        img_np = np.array(img)
        blurred = cv2.GaussianBlur(img_np, (self.kernel_size, self.kernel_size), 0)
        return Image.fromarray(blurred)

def get_transforms(mode=0):
    # ImageNet Normalization
    normalize = T.Normalize([0.485, 0.456, 0.406],
                            [0.229, 0.224, 0.225])

    if mode == 0:
        # Baseline: Resize + Normalize
        train_transform = T.Compose([
            T.Resize((224, 224)),
            T.ToTensor(),
            normalize,
        ])
        val_transform = train_transform

    else:
        # Stage 2: Enhanced with noise reduction, contrast/brightness jitter, and augmentation
        train_transform = T.Compose([
            T.Resize((224, 224)),      # (1) Resize first
            T.RandomHorizontalFlip(),  # (2) Flip
            T.RandomRotation(15),      # (3) Rotate ±15°
            T.ColorJitter(
                brightness=0.1,   # ±10%
                contrast=0.1,     # ±10%
                saturation=0.1,   # ±10%
                hue=0.05          # ±5%
            ),
            T.RandomApply(
                [GaussianBlur(kernel_size=3)],  # (5) Small blur on only some images
                p=0.3
            ),
            T.ToTensor(),
            normalize,
        ])

        val_transform = T.Compose([
            T.Resize((224, 224)),
            T.ToTensor(),
            normalize,
        ])

    return train_transform, val_transform


## e. Main Training Function

In [8]:
# ----------------------------
# Evaluation Function
# ----------------------------
def evaluate(model, loader, device):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            preds = torch.argmax(model(imgs), dim=1)
            correct += (preds == labels).sum().item()
            total   += labels.size(0)
    return correct / total

# ----------------------------
# Train Function
# ----------------------------
def train_model(model, model_name, train_loader, val_loader):
    # Define the model-specific directory under base_dir
    model_root_dir = os.path.join(base_dir, model_name)

    # Auto-versioning inside that model folder
    version = 1
    save_dir = os.path.join(model_root_dir, f"run_v{version}")
    while os.path.exists(save_dir):
        version += 1
        save_dir = os.path.join(model_root_dir, f"run_v{version}")
    os.makedirs(save_dir, exist_ok=True)

    print(f"[{model_name}] Saving checkpoints & plots to: {save_dir}")

    # Training loop as before
    model = model.to(DEVICE)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

    best_acc = 0.0
    train_losses = []
    val_accuracies = []
    epochs_no_improve = 0

    print(f"\n--- Starts Training: {model_name} ---")
    for epoch in range(1, MAX_EPOCHS + 1):
        model.train()
        running_loss = 0.0

        for imgs, labels in train_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(imgs), labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        scheduler.step()

        avg_loss = running_loss / len(train_loader)
        train_losses.append(avg_loss)

        acc = evaluate(model, val_loader, DEVICE)
        val_accuracies.append(acc)

        if acc > best_acc:
            best_acc = acc
            epochs_no_improve = 0
            torch.save(model.state_dict(), os.path.join(save_dir, 'best.pth'))
            print(f"[{model_name}] Epoch {epoch:02d}: ⬆ New best val_acc: {acc:.4f} | Train Loss: {avg_loss:.4f}")
        else:
            epochs_no_improve += 1
            print(f"[{model_name}] Epoch {epoch:02d}: — val_acc did not improve ({acc:.4f}); "
                  f"patience {epochs_no_improve}/{PATIENCE} | Train Loss: {avg_loss:.4f}")

        torch.save(model.state_dict(), os.path.join(save_dir, 'last.pth'))

        if epochs_no_improve >= PATIENCE:
            print(f"Early stopping triggered for {model_name}")
            break

    # Save plots
    epochs = range(1, len(train_losses) + 1)

    plt.figure()
    plt.plot(epochs, train_losses, 'o-', label="Train Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"{model_name} Loss Curve")
    plt.legend(); plt.grid()
    plt.savefig(os.path.join(save_dir, "loss_curve.png")); plt.close()

    plt.figure()
    plt.plot(epochs, val_accuracies, 's-', label="Val Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title(f"{model_name} Accuracy Curve")
    plt.legend(); plt.grid()
    plt.savefig(os.path.join(save_dir, "val_accuracy_curve.png")); plt.close()

## f. Stage Training Functions

### What I Did (Stage 2 Pipeline)
* Gaussian Blur (kernel 5×5) applied to every training and validation image.

* Resize to 224×224 immediately after blurring.

* ColorJitter (brightness ±20 %, contrast ±20 %, saturation ±20 %, hue ±10 %).

* Early Stopping (patience = 5) and checkpoints for “best” and “last” models.

### What I’ll Change (Based on Suggestions)
* Remove Gaussian Blur from Validation

* Keep val_transform = Resize(224,224) → ToTensor → Normalize.

  * This ensures validation images remain sharp and realistic.

* Apply a Milder Blur on Train (Only Occasionally)

* Replace GaussianBlur(kernel_size=5) with RandomApply([GaussianBlur(kernel_size=3)], p=0.3).

* Blur only about 30 % of training images, and use a 3×3 kernel to preserve tumor edges.

* Reorder & Soften Transforms

* New order:

  * Resize(224,224)

  * RandomHorizontalFlip

  * RandomRotation (±15°)

  * ColorJitter (brightness ±10 %, contrast ±10 %, saturation ±10 %, hue ±5 %)

  * Occasional GaussianBlur(3×3)

  * ToTensor → Normalize

  * Smaller jitter reduces the chance of washing out faint tumor‐vs‐tissue contrast.

In [9]:
# ----------------------------
# Stage 1: Baseline Training
# ----------------------------
def stage_1_baseline_training():
    print("\n==== Stage 1: Baseline Training ====\n")

    # Get transforms for Stage 1 (mode=0)
    train_transform, val_transform = get_transforms(mode=0)

    # Create datasets and data loaders
    train_ds = CSVClassificationDataset(TRAIN_CSV, TRAIN_IMG_DIR, transform=train_transform)
    val_ds   = CSVClassificationDataset(VALID_CSV, VALID_IMG_DIR, transform=val_transform)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    # Get all models
    models_list = get_all_models(num_classes=train_ds.num_classes)

    # Loop through models and train each one
    for model_name, model in models_list:
        print(f"\n--- Training {model_name} ---")
        train_model(model, model_name, train_loader, val_loader)

# ----------------------------
# Stage 2: Preprocessing + Augmentation Training
# ----------------------------
def stage_2_preprocessing_training():
    print("\n==== Stage 2: Training with Preprocessing & Augmentation ====\n")

    # Get enhanced transforms (mode=1)
    train_transform, val_transform = get_transforms(mode=1)

    # Prepare datasets and loaders
    train_ds = CSVClassificationDataset(TRAIN_CSV, TRAIN_IMG_DIR, transform=train_transform)
    val_ds   = CSVClassificationDataset(VALID_CSV, VALID_IMG_DIR, transform=val_transform)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    # Load models
    models_list = get_all_models(num_classes=train_ds.num_classes)

    # Train each model
    for model_name, model in models_list:
        print(f"\n--- Stage 2 Training: {model_name} ---")
        train_model(model, model_name, train_loader, val_loader)

# ----------------------------
# Stage 3: Hyperparameter Optimization with Optuna (n_trials per model)
# ----------------------------
def stage_3_hyperparameter_optimization(n_trials=20):
    print(f"\n==== Stage 3: Hyperparameter Search ({n_trials} trials per model) ====\n")

    def objective_for_model(fixed_model_name, trial):
        # 1. Sample hyperparameters (excluding model_name, which is fixed)
        lr           = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
        batch_size   = trial.suggest_categorical("batch_size", [16, 32, 64])
        weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
        dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.5)
        optimizer    = trial.suggest_categorical("optimizer", ["sgd", "adam"])
        momentum     = trial.suggest_float("momentum", 0.5, 0.99)
        scheduler    = trial.suggest_categorical("scheduler", ["none", "steplr", "cosineannealing"])

        # Print trial info
        print(f"\n--- {fixed_model_name} Trial {trial.number + 1}/{n_trials} ---")
        print(f"LR: {lr:.2e} | Batch: {batch_size} | WD: {weight_decay:.2e} | "
              f"Dropout: {dropout_rate:.2f} | Opt: {optimizer} | Momentum: {momentum:.2f} | Scheduler: {scheduler}")

        # 2. Build DataLoaders with stage 3 transforms
        train_transform, val_transform = get_transforms(mode=1)
        train_ds = CSVClassificationDataset(TRAIN_CSV, TRAIN_IMG_DIR, transform=train_transform)
        val_ds   = CSVClassificationDataset(VALID_CSV, VALID_IMG_DIR, transform=val_transform)

        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=2)
        val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=2)

        # 3. Instantiate fixed_model_name from get_all_models and replace dropout
        models_list = get_all_models(train_ds.num_classes)
        model = None
        for name, m in models_list:
            if name == fixed_model_name:
                if fixed_model_name == "ResNet50":
                    m.fc[0] = torch.nn.Dropout(dropout_rate)
                elif fixed_model_name == "EfficientNetB2":
                    m.classifier.insert(0, torch.nn.Dropout(dropout_rate))
                elif fixed_model_name == "MobileNetV3":
                    m.classifier.insert(0, torch.nn.Dropout(dropout_rate))
                model = m.to(DEVICE)
                break

        # 4. Choose optimizer
        if optimizer == "sgd":
            optim_inst = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
        else:
            optim_inst = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

        # 5. Choose scheduler
        if scheduler == "steplr":
            scheduler_inst = optim.lr_scheduler.StepLR(optim_inst, step_size=5, gamma=0.1)
        elif scheduler == "cosineannealing":
            scheduler_inst = optim.lr_scheduler.CosineAnnealingLR(optim_inst, T_max=10)
        else:
            scheduler_inst = None

        criterion = nn.CrossEntropyLoss()
        best_val_acc = 0.0

        # 6. Train for fixed 10 epochs per trial
        for epoch in range(10):
            model.train()
            for imgs, labels in train_loader:
                imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
                optim_inst.zero_grad()
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                loss.backward()
                optim_inst.step()

            if scheduler_inst:
                scheduler_inst.step()

            # 7. Validation
            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for imgs, labels in val_loader:
                    imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
                    outputs = model(imgs)
                    preds = outputs.argmax(dim=1)
                    correct += (preds == labels).sum().item()
                    total += labels.size(0)
            val_acc = correct / total

            # Print epoch progress
            print(f"{fixed_model_name} Trial {trial.number + 1}, Epoch {epoch + 1}/10, Val Acc: {val_acc:.4f}")

            trial.report(val_acc, epoch)
            if trial.should_prune():
                print(f"{fixed_model_name} Trial {trial.number + 1} pruned at epoch {epoch + 1}")
                raise optuna.TrialPruned()

            best_val_acc = max(best_val_acc, val_acc)

        return best_val_acc

    # 8. Loop over each model, run n_trials per model
    all_best_configs = []
    for fixed_model_name in ["ResNet50", "EfficientNetB2", "MobileNetV3"]:
        print(f"\n===== Optimizing {fixed_model_name} =====")
        study = optuna.create_study(direction="maximize")
        func = lambda trial: objective_for_model(fixed_model_name, trial)
        study.optimize(func, n_trials=n_trials)

        # 9. Extract top 3 configs for this model
        df_trials = study.trials_dataframe()
        # Tag every trial with fixed_model_name, since this study only tuned that model
        df_trials["model"] = fixed_model_name
        # Sort all trials by “value” and take the top 3
        df_m_sorted = df_trials.sort_values("value", ascending=False).head(3)

        for rank, row in enumerate(df_m_sorted.itertuples(), start=1):
            all_best_configs.append({
                "model":        row.model,  # same as fixed_model_name
                "rank":         rank,
                "learning_rate": row.params_learning_rate,
                "batch_size":    row.params_batch_size,
                "weight_decay":  row.params_weight_decay,
                "dropout_rate":  row.params_dropout_rate,
                "optimizer":     row.params_optimizer,
                "momentum":      row.params_momentum,
                "scheduler":     row.params_scheduler,
                "val_accuracy":  row.value
            })

    # 10. Save all results to CSV
    df_best = pd.DataFrame(all_best_configs)
    df_best = df_best[[
        "model", "rank", "learning_rate", "batch_size", "weight_decay",
        "dropout_rate", "optimizer", "momentum", "scheduler", "val_accuracy"
    ]]
    summary_path = "/content/drive/MyDrive/Brandon's FYP/hparam_stage3_summary.csv"
    df_best.to_csv(summary_path, index=False)

    print("\n=== Top 3 Configurations per Model ===")
    print(df_best)
    print(f"\nSummary saved to: {summary_path}")

# Main Training Launcher

In [ ]:
# ----------------------------
# Main Launcher
# ----------------------------
if __name__ == '__main__':
    # stage_1_baseline_training()
    # stage_2_preprocessing_training()
    stage_3_hyperparameter_optimization(n_trials=20) # 9 hours, 49 minutes


[I 2025-06-06 18:53:00,682] A new study created in memory with name: no-name-36ac87af-7f93-4066-9cdb-59739e073a86



==== Stage 3: Hyperparameter Search (20 trials per model) ====


===== Optimizing ResNet50 =====

--- ResNet50 Trial 1/20 ---
LR: 3.13e-03 | Batch: 16 | WD: 4.86e-04 | Dropout: 0.32 | Opt: sgd | Momentum: 0.88 | Scheduler: cosineannealing
ResNet50 Trial 1, Epoch 1/10, Val Acc: 0.9530
ResNet50 Trial 1, Epoch 2/10, Val Acc: 0.9775
ResNet50 Trial 1, Epoch 3/10, Val Acc: 0.9826
ResNet50 Trial 1, Epoch 4/10, Val Acc: 0.9777
ResNet50 Trial 1, Epoch 5/10, Val Acc: 0.9867
ResNet50 Trial 1, Epoch 6/10, Val Acc: 0.9902
ResNet50 Trial 1, Epoch 7/10, Val Acc: 0.9883
ResNet50 Trial 1, Epoch 8/10, Val Acc: 0.9889
ResNet50 Trial 1, Epoch 9/10, Val Acc: 0.9894


[I 2025-06-06 19:17:19,287] Trial 0 finished with value: 0.9902253597610643 and parameters: {'learning_rate': 0.0031315785742164507, 'batch_size': 16, 'weight_decay': 0.0004864230118092582, 'dropout_rate': 0.3174454796810835, 'optimizer': 'sgd', 'momentum': 0.8762104225545309, 'scheduler': 'cosineannealing'}. Best is trial 0 with value: 0.9902253597610643.


ResNet50 Trial 1, Epoch 10/10, Val Acc: 0.9889

--- ResNet50 Trial 2/20 ---
LR: 7.69e-05 | Batch: 32 | WD: 4.13e-06 | Dropout: 0.48 | Opt: adam | Momentum: 0.75 | Scheduler: cosineannealing
ResNet50 Trial 2, Epoch 1/10, Val Acc: 0.9661
ResNet50 Trial 2, Epoch 2/10, Val Acc: 0.9772
ResNet50 Trial 2, Epoch 3/10, Val Acc: 0.9851
ResNet50 Trial 2, Epoch 4/10, Val Acc: 0.9883
ResNet50 Trial 2, Epoch 5/10, Val Acc: 0.9886
ResNet50 Trial 2, Epoch 6/10, Val Acc: 0.9867
ResNet50 Trial 2, Epoch 7/10, Val Acc: 0.9894
ResNet50 Trial 2, Epoch 8/10, Val Acc: 0.9905
ResNet50 Trial 2, Epoch 9/10, Val Acc: 0.9889


[I 2025-06-06 19:40:54,488] Trial 1 finished with value: 0.9904968775454792 and parameters: {'learning_rate': 7.690160125267548e-05, 'batch_size': 32, 'weight_decay': 4.130399837032214e-06, 'dropout_rate': 0.4840021733729226, 'optimizer': 'adam', 'momentum': 0.7521860761600443, 'scheduler': 'cosineannealing'}. Best is trial 1 with value: 0.9904968775454792.


...

MobileNetV3 Trial 18, Epoch 1/10, Val Acc: 0.3570
MobileNetV3 Trial 18 pruned at epoch 1

--- MobileNetV3 Trial 19/20 ---
LR: 3.05e-04 | Batch: 16 | WD: 6.14e-05 | Dropout: 0.23 | Opt: adam | Momentum: 0.86 | Scheduler: cosineannealing
MobileNetV3 Trial 19, Epoch 1/10, Val Acc: 0.9226
MobileNetV3 Trial 19, Epoch 2/10, Val Acc: 0.9430
MobileNetV3 Trial 19, Epoch 3/10, Val Acc: 0.9756
MobileNetV3 Trial 19, Epoch 4/10, Val Acc: 0.9794
MobileNetV3 Trial 19, Epoch 5/10, Val Acc: 0.9758
MobileNetV3 Trial 19, Epoch 6/10, Val Acc: 0.9891
MobileNetV3 Trial 19, Epoch 7/10, Val Acc: 0.9859
MobileNetV3 Trial 19, Epoch 8/10, Val Acc: 0.9913
MobileNetV3 Trial 19, Epoch 9/10, Val Acc: 0.9902


[I 2025-06-07 04:40:35,936] Trial 18 finished with value: 0.9923975020363833 and parameters: {'learning_rate': 0.0003048410154682217, 'batch_size': 16, 'weight_decay': 6.13721716330716e-05, 'dropout_rate': 0.22750422326843966, 'optimizer': 'adam', 'momentum': 0.8569593846294994, 'scheduler': 'cosineannealing'}. Best is trial 18 with value: 0.9923975020363833.


MobileNetV3 Trial 19, Epoch 10/10, Val Acc: 0.9924

--- MobileNetV3 Trial 20/20 ---
LR: 3.18e-04 | Batch: 64 | WD: 6.50e-05 | Dropout: 0.22 | Opt: adam | Momentum: 0.88 | Scheduler: none


[I 2025-06-07 04:42:03,674] Trial 19 pruned. 


MobileNetV3 Trial 20, Epoch 1/10, Val Acc: 0.7912
MobileNetV3 Trial 20 pruned at epoch 1

=== Top 3 Configurations per Model ===
            model  rank  learning_rate  batch_size  weight_decay  \
0        ResNet50     1       0.000097          16      0.000070   
1        ResNet50     2       0.000077          32      0.000004   
2        ResNet50     3       0.003132          16      0.000486   
3  EfficientNetB2     1       0.000613          64      0.000006   
4  EfficientNetB2     2       0.000442          64      0.000001   
5  EfficientNetB2     3       0.000364          64      0.000001   
6     MobileNetV3     1       0.000305          16      0.000061   
7     MobileNetV3     2       0.000928          32      0.000062   
8     MobileNetV3     3       0.000072          16      0.000002   

   dropout_rate optimizer  momentum        scheduler  val_accuracy  
0      0.403693      adam  0.534533  cosineannealing      0.994027  
1      0.484002      adam  0.752186  cosineannealing

# 3. Model Testing

## a. Model Evaluation Function




In [10]:
def evaluate_model(model_name, run_version, model_filename):
    print(f"\n--- Evaluating {model_name} ({run_version}) ---")

    # --- Config & paths ---
    DEVICE     = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    BASE_DIR   = "/content/drive/MyDrive/Brandon's FYP"
    MODEL_PATH = os.path.join(BASE_DIR, model_name, run_version, model_filename)
    SAVE_DIR   = os.path.dirname(MODEL_PATH)
    TEST_IMG_DIR = "data/test"
    TEST_CSV     = os.path.join(TEST_IMG_DIR, "_classes.csv")
    os.makedirs(SAVE_DIR, exist_ok=True)

    # --- Load CSV and prepare classes ---
    df = pd.read_csv(TEST_CSV)
    df.columns = df.columns.str.strip()
    class_cols = [c for c in df.columns if c != 'filename']
    NUM_CLASSES = len(class_cols)

    # --- Load model architecture and weights ---
    models_dict = dict(get_all_models(NUM_CLASSES))
    model = models_dict[model_name]
    state = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(state)
    model = model.to(DEVICE)
    model.eval()

    # --- Transforms (match validation pipeline) ---
    transform = T.Compose([
        T.Resize((224,224)),
        T.ToTensor(),
        T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    ])

    # --- Inference loop ---
    y_true, y_pred, y_prob = [], [], []
    with torch.no_grad():
        for _, row in df.iterrows():
            img_path = os.path.join(TEST_IMG_DIR, row['filename'].strip())
            img = Image.open(img_path).convert("RGB")
            x   = transform(img).unsqueeze(0).to(DEVICE)
            logits = model(x)
            probs  = torch.softmax(logits, dim=1).cpu().numpy()[0]
            pred   = int(np.argmax(probs))
            true   = int(row[class_cols].astype(int).values.argmax())
            y_true.append(true)
            y_pred.append(pred)
            y_prob.append(probs)
    y_prob = np.array(y_prob)

    # --- Confusion Matrix ---
    cm = confusion_matrix(y_true, y_pred)
    cm_norm = cm.astype(float) / cm.sum(axis=1)[:, None]
    def plot_cm(matrix, title, fname, norm=False):
        fig, ax = plt.subplots()
        im = ax.imshow(matrix, cmap='Blues', vmin=0 if norm else None, vmax=1 if norm else None)
        plt.colorbar(im, ax=ax)
        ax.set(xticks=np.arange(NUM_CLASSES), yticks=np.arange(NUM_CLASSES),
               xticklabels=class_cols, yticklabels=class_cols,
               xlabel='Predicted', ylabel='Actual', title=title)
        for i in range(NUM_CLASSES):
            for j in range(NUM_CLASSES):
                value = f"{matrix[i,j]:.2f}" if norm else str(matrix[i,j])
                ax.text(j, i, value, ha='center', va='center',
                        color='white' if matrix[i,j] > matrix.max()/2. else 'black')
        fig.tight_layout()
        fig.savefig(os.path.join(SAVE_DIR, fname), dpi=150)
        plt.close(fig)
    plot_cm(cm, f'{model_name}: Confusion Matrix', 'confusion_matrix.png')
    plot_cm(cm_norm, f'{model_name}: Normalized Confusion Matrix', 'confusion_matrix_normalized.png', norm=True)

    # --- F1 Curve ---
    common_t = np.linspace(0,1,100)
    f1_curves = []
    for idx, cls in enumerate(class_cols):
        yb      = np.array(y_true) == idx
        scores  = y_prob[:, idx]
        prec, rec, th = precision_recall_curve(yb, scores)
        f1_vals = 2 * prec * rec / (prec + rec + 1e-8)
        th_ext  = np.concatenate(([0], th))
        f1_curves.append(np.interp(common_t, th_ext, f1_vals))
    mean_f1 = np.mean(f1_curves, axis=0)
    fig, ax = plt.subplots()
    for idx, cls in enumerate(class_cols):
        ax.plot(common_t, f1_curves[idx], label=cls)
    ax.plot(common_t, mean_f1, 'k--', lw=2, label='Average')
    ax.set(title=f'{model_name}: F1 Score vs. Threshold', xlabel='Threshold', ylabel='F1 Score')
    ax.legend(loc='lower left', fontsize='small')
    fig.tight_layout()
    fig.savefig(os.path.join(SAVE_DIR, 'f1_curve.png'), dpi=150)
    plt.close(fig)

    # --- ROC Curve ---
    y_true_bin = label_binarize(y_true, classes=list(range(NUM_CLASSES)))
    fpr, tpr, roc_auc = {}, {}, {}
    for i in range(NUM_CLASSES):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    fig, ax = plt.subplots()
    for i, cls in enumerate(class_cols):
        ax.plot(fpr[i], tpr[i], label=f'{cls} (AUC = {roc_auc[i]:.2f})')
    ax.plot([0,1], [0,1], 'k--', linewidth=1)
    ax.set(xlabel='False Positive Rate', ylabel='True Positive Rate',
           title=f'{model_name}: ROC Curves (OvR)')
    ax.legend(loc='lower right', fontsize='small')
    fig.tight_layout()
    fig.savefig(os.path.join(SAVE_DIR, 'roc_curve.png'), dpi=150)
    plt.close(fig)

    # --- Classification Report ---
    report = classification_report(y_true, y_pred, target_names=class_cols, output_dict=True, zero_division=0)
    df_report = pd.DataFrame(report).T
    df_report.to_csv(os.path.join(SAVE_DIR, 'classification_report.csv'))

    fig, ax = plt.subplots(figsize=(8, 1 + 0.5*len(df_report)))
    ax.axis('off')
    tbl = ax.table(
        cellText=np.round(df_report.values, 3),
        rowLabels=df_report.index,
        colLabels=df_report.columns,
        cellLoc='center', loc='center'
    )
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(10)
    tbl.scale(1, 1.5)
    plt.title(f'{model_name}: Classification Report', pad=20)
    fig.tight_layout()
    fig.savefig(os.path.join(SAVE_DIR, 'classification_report.png'), dpi=150)
    plt.close(fig)

    print(f"✅ Saved all evaluation outputs for {model_name} to:\n → {SAVE_DIR}")

## b. CPU Timing Function

In [51]:
def measure_cpu_inference_time(
    test_csv,
    test_img_dir,
    excel_path,
    sheet_name="Top 3 Configurations",
    output_csv_path="/content/drive/MyDrive/Brandon's FYP/cpu_inference_time_full.csv",
    batch_size=32,
    device=torch.device("cpu")
):

    print("\n=== CPU Inference Measurement START ===")
    print(f"Loading top‐1 configs from '{excel_path}' (sheet '{sheet_name}')…")
    # 1. Load top-1 configs
    df_top3_direct = pd.read_excel(excel_path, sheet_name=sheet_name, header=0)
    df_top1_direct = df_top3_direct[df_top3_direct["Rank"] == 1]
    print(f"Found {len(df_top1_direct)} top-1 configurations (one per model).")

    # 2. Build test DataLoader
    print(f"Building test DataLoader from CSV: '{test_csv}' and image dir: '{test_img_dir}'…")
    _, val_transform = get_transforms(mode=1)
    test_ds = CSVClassificationDataset(test_csv, test_img_dir, transform=val_transform)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=2)
    num_images = len(test_ds)
    num_classes = test_ds.num_classes
    print(f"Test set contains {num_images} images, {num_classes} classes.")

    results = []
    # 3. Measure for each model
    for _, row in df_top1_direct.iterrows():
        model_name = row["Model"]
        dropout_rate = row["Dropout"]
        print(f"\n*** Evaluating model: {model_name} ***")

        # instantiate model
        if model_name == "ResNet50":
            m = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
            m.fc = nn.Sequential(nn.Dropout(dropout_rate),
                                 nn.Linear(m.fc.in_features, num_classes))
        elif model_name == "EfficientNetB2":
            m = models.efficientnet_b2(weights=models.EfficientNet_B2_Weights.IMAGENET1K_V1)
            m.classifier[1] = nn.Sequential(nn.Dropout(dropout_rate),
                                         nn.Linear(m.classifier[1].in_features, num_classes))
        elif model_name == "MobileNetV3":
            m = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.IMAGENET1K_V1)
            m.classifier[3] = nn.Sequential(nn.Dropout(dropout_rate),
                                         nn.Linear(m.classifier[3].in_features, num_classes))
        else:
            print(f"  [Warning] Unknown model '{model_name}', skipping.")
            continue

        m.to(device)
        m.eval()

        # Warm-up
        print("  Warm-up pass…")
        with torch.no_grad():
            imgs, _ = next(iter(test_loader))
            _ = m(imgs.to(device))

        # Timed inference
        print(f"  Timing inference on {num_images} images (batch_size={batch_size})…")
        start = perf_counter()
        with torch.no_grad():
            for imgs, _ in test_loader:
                _ = m(imgs.to(device))
        end = perf_counter()

        elapsed = end - start
        print(f"  ► {model_name} CPU inference time: {elapsed:.2f} s")

        results.append({
            "model": model_name,
            "cpu_inference_time_s": elapsed,
            "num_images": num_images,
            "val_accuracy": row["Val Accuracy"]
        })

    # 4. Output
    df_time = pd.DataFrame(results)
    print("\n=== All models evaluated ===")
    print(df_time)

    print(f"Exporting results to '{output_csv_path}'…")
    df_time.to_csv(output_csv_path, index=False)
    print("Saved CPU timing results successfully.")
    print("=== CPU Inference Measurement END ===\n")


# Main Testing Launcher

In [52]:
if __name__ == '__main__':
    # for model_name in ["ResNet50", "EfficientNetB2", "MobileNetV3"]:
    #     evaluate_model(model_name, run_version="run_v3", model_filename="best.pth")
    measure_cpu_inference_time(
        test_csv=TEST_CSV,
        test_img_dir=TEST_IMG_DIR,
        excel_path=OPTIMIZED_HYPERPARAMETERS
    )


=== CPU Inference Measurement START ===
Loading top‐1 configs from '/content/drive/MyDrive/Brandon's FYP/Hyperparameter Optimization.xlsx' (sheet 'Top 3 Configurations')…
Found 3 top-1 configurations (one per model).
Building test DataLoader from CSV: 'data/test/_classes.csv' and image dir: 'data/test'…
Test set contains 1824 images, 4 classes.

*** Evaluating model: ResNet50 ***
  Warm-up pass…
  Timing inference on 1824 images (batch_size=32)…
  ► ResNet50 CPU inference time: 529.87 s

*** Evaluating model: EfficientNetB2 ***
  Warm-up pass…
  Timing inference on 1824 images (batch_size=32)…
  ► EfficientNetB2 CPU inference time: 270.27 s

*** Evaluating model: MobileNetV3 ***
  Warm-up pass…
  Timing inference on 1824 images (batch_size=32)…
  ► MobileNetV3 CPU inference time: 79.76 s

=== All models evaluated ===
            model  cpu_inference_time_s  num_images  val_accuracy
0        ResNet50            529.872412        1824      0.994027
1  EfficientNetB2            270.26977

# Auto Disconnect After Training

In [53]:
import time
from google.colab import runtime

def disconnect_after(minutes=5):
    print(f"Will disconnect Colab in {minutes} minutes if still running...")
    time.sleep(minutes * 60)
    print("Disconnecting now.")
    runtime.unassign()

if __name__ == '__main__':
    disconnect_after(minutes=5)

Will disconnect Colab in 5 minutes if still running...
Disconnecting now.
